# pandas合并(merge)

In [1]:
import pandas as pd

left = pd.DataFrame({'key':['K0','K1','K2','K3'],
                    'A':['A0','A1','A2','A3'],
                    'B':['B0','B1','B2','B3']})

right = pd.DataFrame({'key':['K0','K1','K2','K3'],
                     'C':['C0','C1','C2','C3'],
                     'D':['D0','D1','D2','D3']})
print(left)
print(right)

  key   A   B
0  K0  A0  B0
1  K1  A1  B1
2  K2  A2  B2
3  K3  A3  B3
  key   C   D
0  K0  C0  D0
1  K1  C1  D1
2  K2  C2  D2
3  K3  C3  D3


## 由column进行合并

merge是对两个DataFrame关于key或者是index进行合并

In [2]:
res = pd.merge(left,right,on='key')
print(res)

  key   A   B   C   D
0  K0  A0  B0  C0  D0
1  K1  A1  B1  C1  D1
2  K2  A2  B2  C2  D2
3  K3  A3  B3  C3  D3


on表示使用哪一个index或者是column进行合并，必须同时存在于左右两个DataFrame对象中

In [3]:
left = pd.DataFrame({'key1':['K0','K0','K1','K2'],
                    'key2':['K0','K1','K0','K1'],
                    'A':['A0','A1','A2','A3'],
                    'B':['B0','B1','B2','B3']})

right = pd.DataFrame({'key1':['K0','K1','K1','K2'],
                     'key2':['K0','K0','K0','K0'],
                     'C':['C0','C1','C2','C3'],
                     'D':['D0','D1','D2','D3']})
print(left)
print(right)

  key1 key2   A   B
0   K0   K0  A0  B0
1   K0   K1  A1  B1
2   K1   K0  A2  B2
3   K2   K1  A3  B3
  key1 key2   C   D
0   K0   K0  C0  D0
1   K1   K0  C1  D1
2   K1   K0  C2  D2
3   K2   K0  C3  D3


In [4]:
res = pd.merge(left,right,on=['key1','key2'])
print(res)

  key1 key2   A   B   C   D
0   K0   K0  A0  B0  C0  D0
1   K1   K0  A2  B2  C1  D1
2   K1   K0  A2  B2  C2  D2


考虑两个key，how有4种值，'left','right','outer','inner'，默认为'inner'内连接

In [5]:
res = pd.merge(left,right,on=['key1','key2'],how='outer') #外连接
print(res)

  key1 key2    A    B    C    D
0   K0   K0   A0   B0   C0   D0
1   K0   K1   A1   B1  NaN  NaN
2   K1   K0   A2   B2   C1   D1
3   K1   K0   A2   B2   C2   D2
4   K2   K1   A3   B3  NaN  NaN
5   K2   K0  NaN  NaN   C3   D3


In [6]:
res = pd.merge(left,right,on=['key1','key2'],how='right') #右外连接
print(res)

  key1 key2    A    B   C   D
0   K0   K0   A0   B0  C0  D0
1   K1   K0   A2   B2  C1  D1
2   K1   K0   A2   B2  C2  D2
3   K2   K0  NaN  NaN  C3  D3


## indicator参数

In [7]:
df1 = pd.DataFrame({'col1':[0,1],'col_left':['a','b']})
df2 = pd.DataFrame({'col1':[1,2,2],'col_right':[2,2,2]})
print(df1)
print(df2)
res = pd.merge(df1,df2,on='col1',how='outer',indicator=True)
print(res)

   col1 col_left
0     0        a
1     1        b
   col1  col_right
0     1          2
1     2          2
2     2          2
   col1 col_left  col_right      _merge
0     0        a        NaN   left_only
1     1        b        2.0        both
2     2      NaN        2.0  right_only
3     2      NaN        2.0  right_only


indicator参数如果为True则向输出DataFrame添加一个名为_merge的列，其中包含关于每行源的信息。通过提供一个字符串参数，可以为列指定一个不同的名称。对于合并键只出现在左侧DataFrame中，值为“left_only”;对于合并键只出现在右侧DataFrame中，值为“right_only”;如果合并键在两个DataFrame中都存在，则值为“both”

In [8]:
res = pd.merge(df1,df2,on='col1',how='outer',indicator='indicator_column')
print(res)

   col1 col_left  col_right indicator_column
0     0        a        NaN        left_only
1     1        b        2.0             both
2     2      NaN        2.0       right_only
3     2      NaN        2.0       right_only


## 由index进行合并

In [9]:
left = pd.DataFrame({'A':['A0','A1','A2'],
                    'B':['B0','B1','B2']},
                    index=['K0','K1','K2'])

right = pd.DataFrame({'C':['C0','C2','C3'],
                     'D':['D0','D2','D3']},
                     index=['K0','K2','K3'])

print(left)
print(right)

     A   B
K0  A0  B0
K1  A1  B1
K2  A2  B2
     C   D
K0  C0  D0
K2  C2  D2
K3  C3  D3


In [10]:
res = pd.merge(left,right,left_index=True,right_index=True,how='outer')
print(res)

      A    B    C    D
K0   A0   B0   C0   D0
K1   A1   B1  NaN  NaN
K2   A2   B2   C2   D2
K3  NaN  NaN   C3   D3


left_index布尔参数，默认为 False。如果为True则使用左侧DataFrame的行索引作为连接键。right_index布尔参数，默认为False。如果为True则使用右侧 DataFrame的行索引作为连接键。

## 处理重叠问题

In [11]:
boys = pd.DataFrame({'k':['K0','K1','K2'],'age':[1,2,3]})
girls = pd.DataFrame({'k':['K0','K0','K3'],'age':[4,5,6]})
print(boys)
print(girls)

    k  age
0  K0    1
1  K1    2
2  K2    3
    k  age
0  K0    4
1  K0    5
2  K3    6


In [12]:
res = pd.merge(boys,girls,on='k',suffixes=['_boy','_girl'],how='inner')
print(res)

    k  age_boy  age_girl
0  K0        1         4
1  K0        1         5


suffixes：一个长度为2的序列，其中每个元素都是一个字符串。当左右DataFrame存在相同列名时，通过该参数可以在相同的列名后增加后缀，默认为('_x','_y')